# Basic imports

In [1]:
import pandas as pd
import os
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
current_path = os.getcwd()
root_path=current_path.replace('\\forward_feature_selection','')

# Loading dataframe

In [3]:
mixed_df=pd.read_csv(root_path+"\\molecules.csv",sep="\t")

# Functions

In [4]:
def save_df_to_disk(df,name:str,separator="\t"):
    df.to_csv(name,sep=separator,index = False, header=True)

In [5]:
def get_data_and_true_prediction(df,not_wanted_features:list):
        temp_df=df.drop(not_wanted_features,axis=1)
        y=temp_df[temp_df.columns[-1]]
        x=temp_df.drop([temp_df.columns[-1]],axis=1)
        
        return x,y   

In [6]:
def get_df_with_name_and_prediction(df,true_prediction,big_df):
    new_df=df
    new_df.insert(0,"m_name",big_df["m_name"].values)
    new_df=new_df.join(true_prediction)
    
    return new_df

# Forward selection

In [7]:
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC

In [8]:
def get_roc_auc_score(x,y,model): # gets roc auc average
        cv_results = cross_validate(model, x, y, cv=10,scoring=('roc_auc'))
        roc_auc_avrg=cv_results['test_score'].mean()
        
        return roc_auc_avrg 

In [9]:
def get_best_score_index(score_lst:list):
    max_score = max(score_lst) # best score
    max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
    return max_score_index[0]

In [10]:
def forward_selection(x,y,model): # O(n) worst case scenario, where n depends on len(x.columns)
    
    def first_iteration(x,y,model):
        score_lst = []
        for i in range(len(x.columns)):
            k = x.columns[i]
            temp_x = x[[k]]
            score = get_roc_auc_score(temp_x,y,model)
            score_lst.append(score)
    
        max_score = max(score_lst) # best score
        max_score_index = [i for i, j in enumerate(score_lst) if j == max_score] # indx with best score
        top_score_index = max_score_index[0]
        new_feature = x.columns[top_score_index] 
        best_x = x[new_feature]
        new_x = x.drop(new_feature,axis=1)
        
        return best_x,new_x,max_score
    
    def else_iteration(best_x,x,y,model,actual_score):     
        new_x_lenght = len(x.columns)
        if (new_x_lenght > 0):
            score_lst = []
            for i in range(new_x_lenght):
                k = x.columns[i]
                temp_x = x[[k]]
                temp_new_x = pd.concat([best_x,temp_x],axis=1, ignore_index=True)
                score = get_roc_auc_score(temp_new_x,y,model)
                score_lst.append(score)

            max_score = max(score_lst) # best score

            if(max_score<actual_score):
                return best_x,actual_score # break condition, recursive function

            max_score_index = [i for i, j in enumerate(score_lst) if j == max_score] # indx with best score
            top_score_index = max_score_index[0]

            new_feature = x.columns[top_score_index]
            temp_x = x[new_feature]
            best_x = pd.concat([best_x,temp_x],axis=1)
            new_x = x.drop(new_feature,axis=1)

            return else_iteration(best_x,new_x,y,model,max_score)
        
        return best_x,actual_score
    
    f_best_x,new_x,f_score = first_iteration(x,y,model)
    best_x,best_score = else_iteration(f_best_x,new_x,y,model,f_score)
    
    return best_x,best_score

In [11]:
def forward_partition_lineal(x,y,lst:list,other): # list must be greater than len 2
    print("\nsecond param list len is {}".format(len(lst)))
    print("second param list -> {}".format(lst))
    
    if (len(lst) == 2):
        first = lst[0]
        print("first is {}".format(first))
        last = lst[-1]
        print("last is {}".format(last))
        clf_first = LinearSVC(random_state=0, tol=first,C=other, dual=False) 
        first_df,first_score = forward_selection(x,y,clf_first)
        clf_last = LinearSVC(random_state=0, tol=last,C=other, dual=False) 
        last_df,last_score = forward_selection(x,y,clf_last)
        
        if (last_score > first_score):
            print("last score is greater than first score")
            return last_df,last,last_score
        
        print("first score is greater than last score")
        return first_df,first,first_score
    
    elif(len(lst) == 3):
        first = lst[0]
        print("first is {}".format(first))
        middle = lst[1]
        print("middle is {}".format(middle))
        last = lst[-1]
        print("last is {}".format(last))
        clf_first = LinearSVC(random_state=0, tol=first,C=other, dual=False) # linear model
        _,first_score = forward_selection(x,y,clf_first)       
        clf_middle = LinearSVC(random_state=0, tol=middle,C=other, dual=False) # linear model
        _,middle_score = forward_selection(x,y,clf_middle)
        clf_last = LinearSVC(random_state=0, tol=last,C=other, dual=False) # linear model
        _,last_score = forward_selection(x,y,clf_last)
        
        if ((first_score >= middle_score) and (last_score <= middle_score)):
            print("sub list taken: left")
            return forward_partition_lineal(x,y,lst[:2],other)  
        
        print("sub list taken: right")            
        return forward_partition_lineal(x,y,lst[1:],other)
    
    else:          
        first = lst[0]
        print("first is {}".format(first))
        middle = lst[(len(lst)//2)-1]
        print("middle is {}".format(middle))
        last = lst[-1]
        print("last is {}".format(last))
        clf_first = LinearSVC(random_state=0, tol=first,C=other, dual=False)
        _,first_score = forward_selection(x,y,clf_first)       
        clf_middle = LinearSVC(random_state=0, tol=middle,C=other, dual=False) 
        _,middle_score = forward_selection(x,y,clf_middle)
        clf_last = LinearSVC(random_state=0, tol=last,C=other, dual=False) 
        _,last_score = forward_selection(x,y,clf_last)
        
        if ((first_score >= middle_score) and (last_score <= middle_score)):
            print("sub list taken: left")
            return forward_partition_lineal(x,y,lst[:(len(lst)//2)],other)
        
        print("sub list taken: right")            
        return forward_partition_lineal(x,y,lst[(len(lst)//2)-1:],other)          

In [12]:
def best_lineal(x,y,K:list,P:list,alfa=6):    
    best_prm_lst = []
    temp_score = 0
    spare_tries = math.floor(len(K)/alfa)+1
    
    for i in K:
        print("----------------------------\n") 
        print("First parameter is {}".format(i))
        best_df,best_p,best_score = forward_partition_lineal(x,y,P,i)  
                  
        print("\n----------------------------")    
        print("When first parameter is {}".format(i))
        print("Best seconds parameter is {}".format(best_p))
        print("ROC AUC = {}".format(best_score))
        print("\nFeatures\n")
        for c,i in enumerate(best_df.columns):
            print("{}. {}".format(c+1,i))
            
        if (temp_score < best_score):
            temp_score = best_score
            spare_tries = math.floor(len(K)/alfa)+1
            print("\nResults improved in last iteration")
        else:
            print("\nResults didn't improve in last iteration")
            spare_tries=spare_tries-1
        
        if(spare_tries <= 0): break      
        best_prm_lst.append([best_df,i,best_p,best_score])
    
    score_lst = [d for [a,b,c,d] in best_prm_lst]
           
    print("\n------------ Final results ----------------")
    index = get_best_score_index(score_lst)
    best_prm = best_prm_lst[index]
    print("Best model when first param = {} ,second param = {} ,ROC AUC = {}".format(best_prm[1],best_prm[2],best_prm[-1]))
    print("\nFeatures\n")
    for c,i in enumerate(best_prm[0].columns):
            print("{}. {}".format(c+1,i))
            
    return best_prm[0]

In [13]:
def guarantee_results(x,y,K:list,P:list):    
    best_outer_prm_lst = []
    for i in K:
        print("First param is {}\n".format(i))
        best_inner_prm_lst = []
        for j in P:
            print("Second param is {}".format(j))
            clf = LinearSVC(random_state=0, tol=j,C=i, dual=False)
            best_df,best_score = forward_selection(x,y,clf)
            print("ROC AUC = {}".format(best_score))
            print("\nFeatures\n")
            for c,d in enumerate(best_df.columns):
                print("{}. {}".format(c+1,d))       
            print("")
            best_inner_prm_lst.append([best_df,i,j,best_score])
            
        score_lst = [d  for [a,b,c,d] in best_inner_prm_lst]
        index = get_best_score_index(score_lst)
        best_outer_prm_lst.append([best_inner_prm_lst[index][0],best_inner_prm_lst[index][1],best_inner_prm_lst[index][2],best_inner_prm_lst[index][3]])
        
    score_lst = [d  for [a,b,c,d] in best_outer_prm_lst]
    index = get_best_score_index(score_lst)
    print("Final results")
    print("Best model when first param = {} ,second param = {} ,ROC AUC = {}".format(best_outer_prm_lst[index][1],
                                                                 best_outer_prm_lst[index][2],
                                                                 best_outer_prm_lst[index][3]))
    
    print("\nFeatures\n")
    for c,d in enumerate(best_outer_prm_lst[index][0].columns):
        print("{}. {}".format(c+1,d))       
            
    return best_outer_prm_lst[index][0]

# Data

In [14]:
unnecesary_features=["m_name"]
x,y = get_data_and_true_prediction(mixed_df,unnecesary_features)

In [15]:
C = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20]
tol = [0.000001,0.00001,0.0001,0.001,0.01,0.1,1]

In [16]:
start = time.time()
best_df = best_lineal(x,y,C,tol)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

----------------------------

First parameter is 1

second param list len is 7
second param list -> [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1]
first is 1e-06
middle is 0.0001
last is 1
sub list taken: left

second param list len is 3
second param list -> [1e-06, 1e-05, 0.0001]
first is 1e-06
middle is 1e-05
last is 0.0001
sub list taken: left

second param list len is 2
second param list -> [1e-06, 1e-05]
first is 1e-06
last is 1e-05
first score is greater than last score

----------------------------
When first parameter is 1
Best seconds parameter is 1e-06
ROC AUC = 0.890847104537271

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_rings
11. n_HOH
12. n_non_strict_rotable_bonds
13. n_atoms_with_Hydrogen
14. n_primary_carbon_atoms
15. n_aromatic_heterocycles
16. n_saturated_heterocycles
17. m_weight
18. m_weigth_without_Hydrogen
19. n_aromatic_carbocycles
2

'00:24:03'

In [ ]:
start = time.time()
g_df = guarantee_results(x,y,C,tol)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

First param is 1

Second param is 1e-06
ROC AUC = 0.890847104537271

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_rings
11. n_HOH
12. n_non_strict_rotable_bonds
13. n_atoms_with_Hydrogen
14. n_primary_carbon_atoms
15. n_aromatic_heterocycles
16. n_saturated_heterocycles
17. m_weight
18. m_weigth_without_Hydrogen
19. n_aromatic_carbocycles
20. n_hetero_cycles

Second param is 1e-05
ROC AUC = 0.8904773002246337

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_rings
11. n_HOH
12. n_non_strict_rotable_bonds
13. n_atoms_with_Hydrogen
14. n_primary_carbon_atoms
15. n_aromatic_heterocycles
16. n_saturated_heterocycles
17. m_avg_weigth
18. n_saturated_carbocycles
19. n_aliphatic_carbocycles

Second param is 0.0001
ROC AUC = 0.88721427848

ROC AUC = 0.8901251486546752

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_rings
11. n_HOH
12. n_non_strict_rotable_bonds
13. n_atoms_without_Hydrogen
14. n_valence_electrons
15. m_avg_weigth
16. n_aromatic_carbocycles
17. n_saturated_carbocycles
18. m_mr

Second param is 0.001
ROC AUC = 0.886908658928365

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_aliphatic_carbocycles
11. n_HOH
12. n_non_strict_rotable_bonds
13. m_avg_weigth

Second param is 0.01
ROC AUC = 0.8874321763044447

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_rings
6. n_amide_bonds
7. n_briged_head_atoms
8. n_strict_rotable_bonds
9. n_saturated_rings
10. n_rings
11. n_HOH
12. n_non_strict_rotable_bonds
13. n_saturated_carbocycles

ROC AUC = 0.8515891339626382

Features

1. n_HBD
2. n_non_strict_rotable_bonds
3. n_aliphatic_heterocycles
4. n_HBA
5. n_aromatic_carbocycles
6. n_hetero_cycles
7. n_aromatic_heterocycles
8. n_amide_bonds
9. n_briged_head_atoms
10. n_saturated_carbocycles
11. fraction_CSP3

First param is 8

Second param is 1e-06


In [ ]:
new_df=get_df_with_name_and_prediction(g_df,y,mixed_df)
save_df_to_disk(new_df,"best_classifier_lineal.csv")

C = 8
Tol = 0.001
ROC AUC = 0.8904

In [ ]:
new_df.head()